# Convolutional Neural Network Example

We build a simple, 2 convolutional and 1 dense layer neural  net and test it on MNIST fashion dataset.


## Preliminaries

### Imports

In [1]:
import pickle

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

import matplotlib.pyplot as plt

%matplotlib inline

import sys
sys.path.append("../..")
import E4525_ML.mnist as mnist
from E4525_ML.TFClassifier import TFClassifier

1.12.0


In [2]:
#
from tensorflow.keras.backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)

sess = tf.Session(config=config)

set_session(sess)  # set this TensorFlow session as the default session for Keras

In [3]:
raw_data_dir="../../raw/fashion"
data_dir="../../data/digits"

In [4]:
images_filename=raw_data_dir+"/train-images-idx3-ubyte.gz"
labels_filename=raw_data_dir+"/train-labels-idx1-ubyte.gz"

test_images_filename=raw_data_dir+"/t10k-images-idx3-ubyte.gz"
test_labels_filename=raw_data_dir+"/t10k-labels-idx1-ubyte.gz"

images=mnist.read_images(images_filename)
labels=mnist.read_labels(labels_filename)

test_images=mnist.read_images(test_images_filename)
test_labels=mnist.read_labels(test_labels_filename)
    
print(images.shape,labels.shape,test_images.shape,test_labels.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


## Keras Model

In [5]:
model = keras.Sequential([
    # keras needs  (Rows x Cols x Channels)
    # we must pass input shape, summaries will not work later
    keras.layers.Reshape((28,28,1),input_shape=(28,28)),
    
    
    # Convolutional Layers
    keras.layers.Conv2D(10, (5, 5), activation='relu',padding="same"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(20, (5, 5), activation='relu',padding="same"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),
    
    # Dense Layers
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [6]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:

print('Trainable parameters: {:,}'.format(model.count_params()))

Trainable parameters: 54,840


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 10)        260       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 20)        5020      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 20)          0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 20)          0         
_________________________________________________________________
flatten (Flatten)            (None, 980)               0         
__________

## Model Training

In [13]:
model.fit(images, labels, epochs=100,validation_data=(test_images,test_labels))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 35s 586us/step - loss: 0.3238 - acc: 0.8805 - val_loss: 0.2904 - val_acc: 0.8968
Epoch 2/100
60000/60000 [==============================] - 34s 574us/step - loss: 0.3021 - acc: 0.8887 - val_loss: 0.2784 - val_acc: 0.8981
Epoch 3/100
60000/60000 [==============================] - 35s 587us/step - loss: 0.2879 - acc: 0.8931 - val_loss: 0.2695 - val_acc: 0.9007
Epoch 4/100
60000/60000 [==============================] - 35s 578us/step - loss: 0.2776 - acc: 0.8968 - val_loss: 0.2585 - val_acc: 0.9053
Epoch 5/100
60000/60000 [==============================] - 35s 579us/step - loss: 0.2674 - acc: 0.9004 - val_loss: 0.2549 - val_acc: 0.9036
Epoch 6/100
60000/60000 [==============================] - 35s 579us/step - loss: 0.2588 - acc: 0.9027 - val_loss: 0.2521 - val_acc: 0.9084
Epoch 7/100
60000/60000 [==============================] - 35s 591us/step - loss: 0.2539 - acc: 0.9043 - val_l

In [14]:
Y_pred=np.argmax(model.predict(test_images),axis=1)

In [15]:
np.mean(Y_pred==test_labels)

0.9166